In [5]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import vstack

In [6]:
import twitter_preproc
import importlib
importlib.reload(twitter_preproc)
from twitter_preproc import *
from operator import attrgetter

In [7]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from twitter_preproc import twitter_preproc
conf = SparkConf().setAll([('spark.executor.memory', '2g'), ('spark.executor.cores', '30'), ('spark.driver.memory','4g')])
spark = SparkSession\
  .builder \
  .appName("Pipelinee") \
  .config(conf=conf) \
  .getOrCreate()
sc = spark.sparkContext

In [8]:
#train = "///user/e11920598/traintweet_1000.tsv"
train = "///tmp/traintweet_1000.tsv"
testfile = "///user/e11920598/test_1000.tsv"
# the full train file has 121.386.431 lines
#train = "///user/pknees/RSC20/training.tsv"
# the full test file has 12.434.838 lines

In [9]:
#    def __init__(self, spark:SparkSession, sc:SparkContext, inputFile:str, seed:int=123,
#                 MF:bool=False, trainsplit:float=0.9):
preproc = twitter_preproc(spark, sc, train)

traindata = preproc.getDF()

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(traindata.take(2), columns=traindata.columns)

In [ ]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

stages = []

stages.append(StringIndexer(inputCol="tweet_id", outputCol="tweet_id_indexed"))
stages.append(OneHotEncoder(inputCol="tweet_id_indexed",  outputCol="tweet_id_ohe"))
stages.append(StringIndexer(inputCol="engaging_user_id", outputCol="user_id_indexed"))
stages.append(OneHotEncoder(inputCol="user_id_indexed",  outputCol="user_id_ohe"))

pipeline = Pipeline(stages=stages)
model = pipeline.fit(traindata)
traindata_ohe = model.transform(traindata).select(["tweet_id_ohe","user_id_ohe","like"])

In [ ]:
df = pd.DataFrame(traindata_ohe.take(1000), columns=traindata_ohe.columns)

In [ ]:
df.head(2)

In [ ]:
def as_matrix(vec):
    data, indices = vec.values, vec.indices
    shape = 1, vec.size
    return csr_matrix((data, indices, np.array([0, vec.values.size])), shape)

In [ ]:
def get_pytorch_sparse(attr):
    features = traindata_ohe.rdd.map(attrgetter(attr))
    mats = features.map(as_matrix)
    mat = mats.reduce(lambda x, y: vstack([x, y]))
    return mat

In [ ]:
tweet_sparse = get_pytorch_sparse("tweet_id_ohe")
user_sparse = get_pytorch_sparse("user_id_ohe")

# Neural Network time :)

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import torch.optim as optim

In [ ]:
# convert to pytorch format
def transform_to_sparse_tensor(data):
    coo = data.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))
    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

In [ ]:
tweet_sparse = transform_to_sparse_tensor(tweet_sparse).to_dense()#.to(torch.device("cpu"))
user_sparse = transform_to_sparse_tensor(user_sparse).to_dense()#.to(torch.device("cpu"))

In [ ]:
y = torch.FloatTensor(traindata_ohe.select("like").collect()) 
target = y  
target = target.view(1, -1).t()

In [ ]:
class Net(nn.Module):

    def __init__(self, users, items, k):
        super(Net, self).__init__()
        self.dense1 = nn.Linear(users.shape[1], k)
        self.dense2 = nn.Linear(items.shape[1], k)
        self.fc1 = nn.Linear(2*k, k)
        self.fc2 = nn.Linear(k, math.floor(k/2))
        self.fc3 = nn.Linear(math.floor(k/2), 1)

    def forward(self, users, items):
        users = F.relu(self.dense1(users))
        items = F.relu(self.dense2(items))
        # concat users and items into 1 vector
        x = torch.cat((users, items), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        sigfried = nn.Sigmoid()
        x = sigfried(self.fc3(x))
        return x

In [ ]:
net = Net(user_sparse, tweet_sparse, 100)
print(net)

In [ ]:
output = net(user_sparse, tweet_sparse)
#target = y  
#target = target.view(1, -1).t()  # make it the same shape as output
criterion = nn.BCELoss()

loss = criterion(output, target)
print(loss)

In [ ]:
#net.zero_grad()     # zeroes the gradient buffers of all parameters
#loss.backward()

In [ ]:
# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(user_sparse, tweet_sparse)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

In [ ]:
1+1

In [ ]:
n_epochs = 5 # or whatever
batch_size = 128 # or whatever
criterion = nn.BCELoss()
optimizer = optim.SGD(net.parameters(), lr=0.01)

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(user_sparse.size()[0])

    for i in range(0,user_sparse.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x_user = user_sparse[indices]
        batch_x_tweet = tweet_sparse[indices]
        batch_y = target[indices]
        exit(0)

        # in case you wanted a semi-full example
        outputs = net.forward(batch_x_user, batch_x_tweet)
        loss = criterion(outputs,batch_y)

        loss.backward()
        optimizer.step()
        print(loss)